In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

In [8]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "https://morepower.com.ph/monthly-rates/"
pdf_folder = "pdf downloads"

# Create the folder if it doesn't exist
os.makedirs(pdf_folder, exist_ok=True)

# Define headers to include in the request
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the webpage with headers
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all links that end with .pdf
    pdf_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
    
    # Download each PDF
    for link in pdf_links:
        pdf_url = urljoin(url, link)  # Create the full URL if it's relative
        pdf_filename = os.path.join(pdf_folder, os.path.basename(pdf_url))
        
        # Send a GET request to the PDF URL
        pdf_response = requests.get(pdf_url, headers=headers)
        
        if pdf_response.status_code == 200:
            # Save the PDF content
            with open(pdf_filename, "wb") as pdf_file:
                pdf_file.write(pdf_response.content)
            print(f"Downloaded {pdf_filename}")
        else:
            print(f"Failed to download {pdf_url}. Status code: {pdf_response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Downloaded pdf downloads/Generation-Charge_July2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_June-2024_Web-Upload_revised0618.pdf
Downloaded pdf downloads/Generation-Charge_May-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Apr-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Mar-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Feb-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Jan-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Dec-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Nov-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Oct-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Sep-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_August-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_July-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_June-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge-f

### **Processing**

In [41]:
#df = tabula.read_pdf("pdf downloads/GenCharge_May2020.pdf", lattice = True, pages = 'all')[0]

# Path to the PDF file
pdf_path = "pdf downloads/GenCharge_May2020.pdf"

# Read all tables from the specified page(s)
tables = tabula.read_pdf(pdf_path, lattice=True, pages='all', multiple_tables=True)

# Ensure there are at least two tables
if len(tables) < 2:
    raise ValueError("There are fewer than 2 tables in the PDF.")

# Process the first table (assuming it has headers)
first_table = tables[0]

# Process the second table (assuming it should include its headers as a row)
second_table = tables[1]

# Extract column names from the first table
first_table_columns = first_table.columns

# Convert the header row of the second table to a DataFrame row
second_table_header_as_row = pd.DataFrame([second_table.columns.tolist()], columns=second_table.columns)

# Append the header row to the second table data
second_table_with_header_as_row = pd.concat([second_table_header_as_row, second_table], ignore_index=True)

# Rename columns of the second table to match the first table
second_table_with_header_as_row.columns = first_table_columns

# Combine the first table with the modified second table
combined_df = pd.concat([first_table, second_table_with_header_as_row], ignore_index=True)

combined_df

,GENERATION CHARGE for MAY 2020,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Power Supplier,% Share on\rEnergy\rPurchased,Energy\rPurchased\rkWh,Basic Generation\rCost\rPhP,Other Cost\rAdjustment\rPhP,Total Generation\rCost for the Month\rPhP,Average\rGeneration Rate\rPhP/KWH
1,NaN,(A),(B),(C),(D = B+C),(D/A),NaN
2,BILATERAL CONTRACTS,NaN,NaN,NaN,NaN,NaN,NaN
3,1. Panay Energy Development Corporation,76.01%,"38,658,046.66","257,968,168.15",-,"257,968,168.15",6.6731
4,2. Panay Power Corporation,0.40%,"204,000.00","9,788,428.80",-,"9,788,428.80",47.9825
5,3. KEPCO SPC Power Corporation,7.08%,"3,600,000.00","16,833,436.13",-,"16,833,436.13",4.6760
6,"4. Aboitiz Power Renewables, Incorporated",0.00%,-,-,-,-,-
7,Subtotal - Emergency Power Supply Contracts,83.49%,"42,462,046.66","284,590,033.08",-,"284,590,033.08",6.7022
8,Wholesale Electricity Spot Market (WESM),16.51%,"8,394,227.66","15,627,992.12","833,614.18","16,461,606.30",1.9611
9,Export Energy from Net Metering Customers,0.00%,-,-,-,-,-
